# See input.sx for a sample SPHInX input that I would like to achieve

In [71]:
from pyiron import Project
from pyiron.sphinx.structure import read_atoms

In [115]:
pr = Project("sphinx_input")
test = pr.create_job(pr.job_type.Sphinx, "test")
test.structure = read_atoms("structure.sx")

2020-04-07 15:49:02,792 - pyiron_log - WARNING - Job aborted - please remove it and run again! test


# Currently:
What I have below doesn't work. It ultimately writes the _guess_str dictionary literally to guess.sx.
Normally I just overwrite the file, e.g. `with open("guess.sx", "w")`... immediately
after I submit the job, which cannot be the best way.

In [116]:
test._guess_str = {
    "waves": {
        "lcao": {
            "maxSteps": 1,
            "rhoMixing": 0.
        },
        "pawBasis": None
    },
    "rho": {
        "atomicOrbitals": None,
        "charged": {
            "charge": 2,
            "z": 20
        }
    }
}

#test.run()

# It would be great to arbitrarily add/overwrite parameters with dictionaries:

I think if the primary groups that should be tunable in SPHInX (basis, PAWHamiltonian, and initialGuess) were initialized as full dictionaries with their respective defaults, this would be a straightforward way to customize them later, as Sam suggested.

And if they are meant to be externally modifiable, they should probably not carry an underscore prefix.

In [32]:
class SphinxBase():
    """
    Class to setup and run Sphinx simulations which is a derivative of pyiron_atomistics.job.generic.GenericJob.
    The functions in these modules are written in such the function names and attributes are very generic
    (get_structure(), molecular_dynamics(), version) but the functions are written to handle Sphinx specific input and
    output.
    Args:
        project: Project object (defines path where job will be created and stored)
        job_name (str): name of the job (must be unique within this project path)
    """

    def __init__(self):
        self.basis_group = {
            "eCut": 340,
            "kPoint": {
                "coords": [1/2, 1/2, 1/4],
                "weight": 1,
                "relative": True
            },
            "folding": [5, 5, 1]
        }
        self.hamilton_group = {
            "nEmptyStates": "auto",
            "ekt": 0.1,
            "xc": "PBE"
        }
        self.guess_group = {
            "waves": {
                "lcao": {
                    "maxSteps": 1,
                    "rhoMixing": 0.
                },
                "pawBasis": True
            },
           "rho": {
               "atomicOrbitals": True,
           }
        }
        self.main_group = {}  # May be no sensible defaults
        self.spin_group = None  # This group is optional
        self.structure_group = None # Can't be initialized w/o structure
        self.species_group = None  # Can't be initialized w/o structure

In [111]:
sphinx_base = SphinxBase()
sphinx_base.hamilton_group["dipoleCorrection"] = None
sphinx_base.hamilton_group["nExcessElectrons"] = -2
sphinx_base.guess_group["charged"] = {"charge": 2, "z": 20}